In [ ]:
import pandas as pd

# 读取附件1中“乡村的现有耕地”表
df_land = pd.read_excel("附件1.xlsx", sheet_name="乡村的现有耕地")
df_land.columns = df_land.columns.str.strip()  # 清理列名中的空格

# 读取附件1中“乡村种植的农作物”表
df_crops = pd.read_excel("附件1.xlsx", sheet_name="乡村种植的农作物")
df_crops.columns = df_crops.columns.str.strip()  # 清理列名中的空格

# 读取附件2中“2023年的农作物种植情况”表
df_planting_2023 = pd.read_excel("附件2_1.xlsx", sheet_name="2023年的农作物种植情况")
df_planting_2023.columns = df_planting_2023.columns.str.strip()  # 清理列名中的空格

# 读取附件2中“2023年统计的相关数据”表
df_stats_2023 = pd.read_excel("附件2_1.xlsx", sheet_name="2023年统计的相关数据")
df_stats_2023.columns = df_stats_2023.columns.str.strip()  # 清理列名中的空格


# print(df_land.columns)
# print(df_crops.columns)
# print(df_planting_2023.columns)
# print(df_stats_2023.columns)

# 尝试使用其他可能的列进行合并
df = pd.merge(df_planting_2023, df_land, on="地块名称")  # 改为使用 "种植地块" 列进行合并
df = pd.merge(df, df_crops, on="作物编号")
df = pd.merge(df, df_stats_2023, on=["作物编号", "地块类型", "种植季次"])

# 数据清洗
# 检查缺失值
print(df.isnull().sum())

# 数据转换
# 计算销售价格的平均值
# df["销售单价"] = df["销售单价"].apply(lambda x: (x[0] + x[1]) / 2)
df[["最低价", "最高价"]] = (
    df["销售单价/(元/斤)"].str.split("-", expand=True).astype(float)
)
df["平均价"] = (df["最低价"] + df["最高价"]) / 2
df = df.drop(["销售单价/(元/斤)", "说明_x", "说明_y","种植耕地", "序号"], axis=1)
df.columns = df.columns.str.split('_').str[0]

df["预计销售量"] = df["种植面积/亩"]*df["亩产量/斤"]
print(df.columns)
# 特征工程
# 计算每种作物的总种植面积
# df["总种植面积"] = df.groupby("作物名称")["种植面积"].transform("sum")

# 保存预处理后的数据
df.to_excel("预处理后的数据.xlsx", index=False)

In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, lpDot, value, LpStatus

# 读取预处理后的数据
data = pd.read_excel("预处理后的数据.xlsx")

# 确保数据中所有需要的列都有值
required_columns = ["作物名称", "平均价", "亩产量/斤", "种植成本/(元/亩)", "作物类型"]
if not all(col in data.columns for col in required_columns):
    raise ValueError("数据缺少必要的列")

# 假设预计销售量相对于 2023 年保持稳定
data["预计销售量"] = data.groupby("作物名称")["种植面积/亩"].transform("sum")

# 提取相关数据
years = range(2024, 2031)
crops = data["作物名称"].unique()
plots = data["地块名称"].unique()

# 定义问题
prob1_1 = LpProblem("Problem_1_1", LpMaximize)
prob1_2 = LpProblem("Problem_1_2", LpMaximize)

# 定义变量
x1_1 = LpVariable.dicts("x1_1", [(i, j, t) for i in plots for j in crops for t in years], lowBound=0, cat='Continuous')
x1_2 = LpVariable.dicts("x1_2", [(i, j, t) for i in plots for j in crops for t in years], lowBound=0, cat='Continuous')

# 设定目标函数
for j in crops:
    for t in years:
        crop_data = data.loc[data["作物名称"] == j].iloc[0]
        profit_per_unit = crop_data["平均价"] * crop_data["亩产量/斤"] - crop_data["种植成本/(元/亩)"]
        prob1_1 += lpDot([x1_1[i, j, t] for i in plots], profit_per_unit)

        total_production = lpDot([x1_2[i, j, t] for i in plots], crop_data["亩产量/斤"])
        excess = total_production - crop_data["预计销售量"]
        prob1_2 += profit_per_unit * total_production - 0.5 * crop_data["平均价"] * excess

# 设定约束条件
# 耕地资源约束
for t in years:
    prob1_1 += lpSum([x1_1[i, j, t] for i in plots for j in crops]) <= 1201
    prob1_2 += lpSum([x1_2[i, j, t] for i in plots for j in crops]) <= 1201

# 种植间作的要求：同一地块同一季节不能种植两种粮食类作物
for i in plots:
    for t in years:
        grain_crops = [j for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食")]
        prob1_1 += lpSum([x1_1[i, j, t] for j in grain_crops]) <= 1
        prob1_2 += lpSum([x1_2[i, j, t] for j in grain_crops]) <= 1

# 豆类作物轮作要求：每个地块三年内至少种植一次豆类作物
for i in plots:
    for t in years:
        if t <= 2026:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）")]) >= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）")]) >= 1

for i in plots:
    if data.loc[data["地块名称"] == i, "地块类型"].values[0] == "水浇地":
        for t in years:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if j == "水稻" or data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("蔬菜")]) <= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if j == "水稻" or data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("蔬菜")]) <= 1
    else:
        for t in years:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食") and j!= "水稻"]) <= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食") and j!= "水稻"]) <= 1

# 提升非豆类作物的最低种植面积
for j in crops:
    for t in years:
        if not data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）"):
            prob1_1 += lpSum([x1_1[i, j, t] for i in plots]) >= 30  # 设定非豆类作物最低种植面积
            prob1_2 += lpSum([x1_2[i, j, t] for i in plots]) >= 30  # 设定非豆类作物最低种植面积
for j in crops:
    for t in years:
        if not data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食"):
            prob1_1 += lpSum([x1_1[i, j, t] for i in plots]) >= 20  # 设定非豆类作物最低种植面积
            prob1_2 += lpSum([x1_2[i, j, t] for i in plots]) >= 20  # 设定非豆类作物最低种植面积

# 确保每个地块每年都有种植作物
for i in plots:
    for t in years:
        prob1_1 += lpSum([x1_1[i, j, t] for j in crops]) >= 0.6  # 确保每个地块每年都至少种植0.3亩作物
        prob1_2 += lpSum([x1_2[i, j, t] for j in crops]) >= 0.6  # 确保每个地块每年都至少种植0.3亩作物

# 求解问题
prob1_1.solve()
prob1_2.solve()

# 检查求解器的状态
print("Status for Problem 1_1:", LpStatus[prob1_1.status])
print("Status for Problem 1_2:", LpStatus[prob1_2.status])

# 提取结果
result1_1 = pd.DataFrame([(i, j, t, value(x1_1[i, j, t])) for i in plots for j in crops for t in years if value(x1_1[i, j, t]) > 0], columns=["地块名称", "作物名称", "年份", "种植面积/亩"])
result1_2 = pd.DataFrame([(i, j, t, value(x1_2[i, j, t])) for i in plots for j in crops for t in years if value(x1_2[i, j, t]) > 0], columns=["地块名称", "作物名称", "年份", "种植面积/亩"])

two_season = ["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8",
                    "E1", "E2", "E3", "E4", "E5", "E6", "E7", "E8",
                    "E9", "E10", "E11", "E12", "E13", "E14", "E15", "E16",
                    "F1", "F2", "F3", "F4"]

# 确保地块名称的列名正确
result1_1 = result1_1.rename(columns={"地块名称": "地块名称"})

# 筛选出包含 two_season 中地块名称的行
result1_1_two_season = result1_1[result1_1["地块名称"].isin(two_season)]

# 分离出第一季和第二季的数据
season1_df = result1_1_two_season[result1_1_two_season["地块名称"].isin(two_season) & (result1_1_two_season["年份"] % 2 == 0)].copy()
season1_df['季节'] = "第一季"

season2_df = result1_1_two_season[result1_1_two_season["地块名称"].isin(two_season) & (result1_1_two_season["年份"] % 2 == 1)].copy()
season2_df['季节'] = "第二季"

# 合并第一季和第二季的数据
result1_1_two_season = pd.concat([season1_df, season2_df])

# 处理其他地块的第一季数据
season1_df_other = result1_1[~result1_1["地块名称"].isin(two_season)].copy()
season1_df_other['季节'] = "第一季"

# 合并所有数据
result1_1 = pd.concat([season1_df_other, result1_1_two_season])
result1_1.reset_index(drop=True, inplace=True)

# 检查每行之和是否为 0.6
for index, row in result1_1.iterrows():
    total_area = row.drop(['地块名称', '作物名称', '年份', '季节']).sum()
    if total_area!= 0.6:
        print(f"Row {index} does not sum to 0.6. Sum is {total_area}.")
# 按照要求整理结果
result1_1_formatted = {}
result1_2_formatted = {}
for t in years:
    result1_1_formatted[t] = pd.DataFrame(0, index=plots, columns=crops)
    result1_2_formatted[t] = pd.DataFrame(0, index=plots, columns=crops)

for _, row in result1_1.iterrows():
    plot_name = row["地块名称"]
    crop_name = row["作物名称"]
    year = row["年份"]
    planting_area = row["种植面积/亩"]

    result1_1_formatted[year].loc[plot_name, crop_name] = planting_area

for _, row in result1_2.iterrows():
    plot_name = row["地块名称"]
    crop_name = row["作物名称"]
    year = row["年份"]
    planting_area = row["种植面积/亩"]

    result1_2_formatted[year].loc[plot_name, crop_name] = planting_area

# 保存结果
with pd.ExcelWriter("result1_1.xlsx") as writer:
    for t, df in result1_1_formatted.items():
        df.to_excel(writer, sheet_name=str(t))

with pd.ExcelWriter("result1_2.xlsx") as writer:
    for t, df in result1_2_formatted.items():
        df.to_excel(writer, sheet_name=str(t))

In [ ]:
import numpy as np
from scipy.optimize import linprog

# 定义数据
crops = [
    ('黄豆', 400, 400, 2.50, 4.00),
    ('黑豆', 500, 400, 6.50, 8.50),
    ('红豆', 400, 350, 7.50, 9.00),
    ('绿豆', 350, 350, 6.00, 8.00),
    ('爬豆', 415, 350, 6.00, 7.50),
    ('小麦', 800, 450, 3.00, 4.00),
    ('玉米', 1000, 500, 2.50, 3.50),
    ('谷子', 400, 360, 6.00, 7.50),
    ('高粱', 630, 400, 5.50, 6.50),
    ('黍子', 525, 360, 6.50, 8.50),
    ('荞麦', 110, 350, 30.00, 50.00),
    ('南瓜', 3000, 1000, 1.00, 2.00),
    ('红薯', 2200, 2000, 2.50, 4.00),
    ('莜麦', 420, 400, 5.00, 6.00),
    ('大麦', 525, 350, 3.00, 4.00),
    ('水稻', 500, 680, 6.00, 8.00),
    ('豇豆', 3000, 2000, 7.00, 9.00),
    ('刀豆', 2000, 1000, 5.50, 8.00),
    ('芸豆', 3000, 2000, 5.00, 8.00),
    ('土豆', 2000, 2000, 3.00, 4.50),
    ('西红柿', 2400, 2000, 5.00, 7.50),
    ('茄子', 6400, 2000, 5.00, 6.00),
    ('菠菜', 2700, 2300, 4.80, 6.70),
    ('青椒', 2400, 1600, 4.00, 6.50),
    ('菜花', 3300, 2400, 5.00, 6.00),
    ('包菜', 3700, 2900, 5.50, 7.50),
    ('油麦菜', 4100, 1600, 4.00, 6.00),
    ('小青菜', 3200, 1600, 5.00, 6.50),
    ('黄瓜', 12000, 2900, 6.00, 8.00),
    ('生菜', 4100, 1600, 4.50, 6.00),
    ('辣椒', 1600, 1000, 6.00, 8.50),
    ('空心菜', 10000, 4100, 3.00, 6.00),
    ('黄心菜', 5000, 2000, 4.00, 5.00),
    ('芹菜', 5500, 900, 3.20, 4.80),
    ('榆黄菇', 5000, 3000, 50.00, 65.00),
    ('香菇', 4000, 2000, 18.00, 20.00),
    ('白灵菇', 10000, 10000, 14.00, 18.00),
    ('羊肚菌', 1000, 10000, 80.00, 120.00)
]

# 定义作物数量
num_crops = len(crops)

# 定义地块和大棚
num_plots = 34
num_greenhouses = 16
num_smart_greenhouses = 4

# 每块地的面积（亩）
plot_area = np.array([1201 / num_plots] * num_plots + [0.6] * (num_greenhouses + num_smart_greenhouses))

# 设置目标函数
c = np.zeros(num_crops)
for i in range(num_crops):
    avg_price = (crops[i][3] + crops[i][4]) / 2
    c[i] = avg_price * crops[i][1]  # 平均销售价格 * 亩产量

# 设定线性规划问题
# 目标函数：最大化总收入
f = -c  # 线性规划默认最小化目标函数，因此我们取负值以实现最大化

# 约束条件
A = -np.eye(num_crops)
b = -plot_area[:num_crops]  # 最大种植面积

# Bounds should only apply to the decision variables (crop areas)
bounds = [(0, plot_area[i]) for i in range(num_crops)]

# 线性规划求解
result = linprog(f, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# 结果
print('最优种植方案：')
for i in range(num_crops):
    print(f'作物 {crops[i][0]}: {result.x[i]:.2f} 亩')

print(f'总收益: {-result.fun:.2f} 元')


: 